基于李沐大神的b站课程

https://www.bilibili.com/video/BV1if4y147hS?spm_id_from=333.999.0.0

# 数据预处理
## ？？创建一个人工数据集，并存储在csv文件
<font color = 'red'>p47 os操作不会</font>

In [4]:
import os

In [18]:
os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms, Alley, price\n') # 列名
    f.write('NA,Pave,127500\n')
    f.write('2,NA,10600\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [27]:
import pandas as pd

data = pd.read_csv(data_file)
data

,NumRooms,Alley,price
0,NaN,Pave,127500
1,2.0,NaN,10600
2,4.0,NaN,178100
3,NaN,NaN,140000


## 处理缺失数据
### 插值、删除

* `df.fillna(df.mean())`填平均值
* `pd.get_dummies(df, dummy_na = True)`对于离散化值（包括`NaN`）one hot encode

In [36]:
inputs, outputs = data.iloc[: , 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean())
inputs

,NumRooms,Alley
0,3.0,Pave
1,2.0,NaN
2,4.0,NaN
3,3.0,NaN


In [38]:
inputs = pd.get_dummies(inputs, dummy_na=True) # one hot encode
inputs

,NumRooms,Alley_Pave,Alley_nan
0,3.0,1,0
1,2.0,0,1
2,4.0,0,1
3,3.0,0,1


float64比较慢，深度学习一般会改为32位浮点数

In [50]:
import torch

X,y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X,y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500,  10600, 178100, 140000]))

# 线性代数
* 分配新内存

In [59]:
a = np.arange(10).reshape((2,5))
b = a
b[:] = 1
b,a

(array([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]]),
 array([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]]))

In [57]:
a = np.arange(10).reshape((2,5))
b = a.copy()
b[:] = 1
b,a

(array([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]]),
 array([[0, 1, 2, 3, 4],
        [5, 6, 7, 8, 9]]))

In [63]:
A = torch.arange(20, dtype=torch.float32).reshape(5, 4)
B = A
A, A + B

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.],
         [16., 17., 18., 19.]]),
 tensor([[ 0.,  2.,  4.,  6.],
         [ 8., 10., 12., 14.],
         [16., 18., 20., 22.],
         [24., 26., 28., 30.],
         [32., 34., 36., 38.]]))

In [62]:
A = torch.arange(20, dtype=torch.float32).reshape(5, 4)
B = A.clone()
A, A + B

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.],
         [16., 17., 18., 19.]]),
 tensor([[ 0.,  2.,  4.,  6.],
         [ 8., 10., 12., 14.],
         [16., 18., 20., 22.],
         [24., 26., 28., 30.],
         [32., 34., 36., 38.]]))

In [66]:
A.numel() # 元素总个数
A.shape[0]

5

计算总和或均值时保持轴数不变——keepdims=True
* 为了更好的使用广播机制
`A/sum_A`

In [71]:
A = torch.arange(20, dtype=torch.float32).reshape(5, 4)
sum_A = A.sum(axis = 1)
sum_A,A

(tensor([ 6., 22., 38., 54., 70.]),
 tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.],
         [16., 17., 18., 19.]]))

In [68]:
A = torch.arange(20, dtype=torch.float32).reshape(5, 4)
sum_A = A.sum(axis = 1, keepdims=True)
sum_A,A

(tensor([[ 6.],
         [22.],
         [38.],
         [54.],
         [70.]]),
 tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [12., 13., 14., 15.],
         [16., 17., 18., 19.]]))

In [76]:
A = torch.arange(20, dtype=torch.float32).reshape(5, 4)
A.cumsum(axis=0) #  按轴累加

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  6.,  8., 10.],
        [12., 15., 18., 21.],
        [24., 28., 32., 36.],
        [40., 45., 50., 55.]])

* 矩阵乘法【点积】`torch.mv(A,x)`——【matrix-vector multiplication】
* 矩阵乘法`torch.mm(A.T,A)`

In [82]:
A

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.],
        [12., 13., 14., 15.],
        [16., 17., 18., 19.]])

In [81]:
A.T

tensor([[ 0.,  4.,  8., 12., 16.],
        [ 1.,  5.,  9., 13., 17.],
        [ 2.,  6., 10., 14., 18.],
        [ 3.,  7., 11., 15., 19.]])

In [91]:
A = torch.arange(20, dtype=torch.float32).reshape(5, 4)
x = torch.arange(4,dtype = torch.float32)
A@x, torch.mv(A,x) == A@x


(tensor([ 14.,  38.,  62.,  86., 110.]),
 tensor([True, True, True, True, True]))

In [92]:
A = torch.arange(20, dtype=torch.float32).reshape(5, 4)
torch.mm(A,A.T),torch.mm(A,A.T) == A@A.T

(tensor([[  14.,   38.,   62.,   86.,  110.],
         [  38.,  126.,  214.,  302.,  390.],
         [  62.,  214.,  366.,  518.,  670.],
         [  86.,  302.,  518.,  734.,  950.],
         [ 110.,  390.,  670.,  950., 1230.]]),
 tensor([[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]))

范数torch.norm()

In [94]:
u = torch.tensor([3.0, -4.0])
torch.norm(u),torch.abs(u).sum()

(tensor(5.), tensor(7.))

按某一维度求和维度保留问题

In [97]:
a = torch.ones((2, 5, 4))
a.shape,a

(torch.Size([2, 5, 4]),
 tensor([[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]],
 
         [[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]))

In [99]:
a.sum(axis=1).shape,a.sum(axis=1)

(torch.Size([2, 4]),
 tensor([[5., 5., 5., 5.],
         [5., 5., 5., 5.]]))

In [101]:
a.sum(axis=[0,2]).shape,a.sum(axis = [0,2])

(torch.Size([5]), tensor([8., 8., 8., 8., 8.]))

In [102]:
a.sum(axis=1,keepdims=True).shape,a.sum(axis=1,keepdims=True)

(torch.Size([2, 1, 4]),
 tensor([[[5., 5., 5., 5.]],
 
         [[5., 5., 5., 5.]]]))